In [1]:
#Datos sobre la base de datos configurada. Ponerlo como variable de sistema para no ser publicados mis datos

config = {
    "host":"database-1.cr4emq2gye48.eu-west-3.rds.amazonaws.com",
    "user":"postgres",
    "password": "Hola1234",
    "port": 5432,
    "dbname":"postgres"
    }

In [2]:
#Ahora conectamos a base de datos 

import psycopg2

try: #en caso de que se conecte salga esto
    conn = psycopg2.connect(**config)
    print("Conectado con éxito!")
    conn.close()
except Exception as e: # si no se conecta sale esto
    print("No conectados, algo está saliendo mal")
#Esto es para comprobar que se haya conectado correctamente

Conectado con éxito!


Creamos otra conexión

In [3]:
#Funcion que pone la pregunta y la respuesta en la bbdd

from datetime import datetime
def bbdd (pregunta, respuesta):
    conn = psycopg2.connect(**config)
    cursor = conn.cursor()

    query =  f"INSERT INTO preguntas_respuestas (preguntas, respuesta, fecha) VALUES (%s, %s, %s);"

    cursor.execute(query, (pregunta, respuesta, datetime.now())) #se le pasa los datto en tupla, para ser añadidos en la base de datos

    conn.commit() #Siempre que hay un insert se pone un commit
    cursor.close() #Cerramos el cursos 
    conn.close() 

    return "Ok"

In [4]:
pregunta = "Cómo te apellidas?"
respuesta = "Alonso"

resp = bbdd(pregunta, respuesta)
respuesta

'Alonso'

## API Groc LLM

In [5]:
import os
os.environ["GROQ_API_KEY"] = "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"

In [ ]:

from groq import Groq
def llm(pregunta):
  client = Groq(
      api_key=os.environ.get("GROQ_API_KEY"),
  )

  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "system", #revisar cómo usar esto
              "content": """Eres una IA especializada **exclusivamente** en crear rutinas de entrenamiento personalizadas.  

                Tu función única: diseñar planes de ejercicio adaptados a las necesidades que te indique el usuario (objetivo, deporte, días por semana, tiempo disponible, nivel, material, limitaciones no médicas).  

                Temas permitidos  
                - Rutinas de fuerza, hipertrofia, resistencia, movilidad, HIIT, deporte específico.  
                - Selección de ejercicios, series, repeticiones, descansos, progresiones/regresiones.  
                - Distribución semanal, duración de sesiones, calentamientos y estiramientos.  
                - Adaptación según material disponible (gimnasio, mancuernas, bandas, peso corporal).  

                Temas prohibidos (rechaza de forma graciosa):  
                - Nutrición, dietas, calorías o suplementos.  
                - Consejos médicos, diagnósticos, tratamientos, embarazo, posparto.  
                - Política, economía, temas no relacionados con entrenamiento.  

                Seguridad
                No des consejos médicos. Si el usuario menciona dolor, lesión o condición médica, responde con un recordatorio breve:  
                > “Detén el ejercicio y consulta a un profesional sanitario antes de continuar.”  

                Estructura de respuesta obligatoria  
                1.Resumen del plan (objetivo, frecuencia, duración).  
                2.Calendario semanal (qué se entrena cada día).  
                3.Sesiones detalladas (ejercicios con series × repeticiones, descansos, RPE/RIR).  
                4.Progresión (cómo avanzar semana a semana).  
                5.Sustituciones (si falta material o hay molestias).  
                6.Indicaciones técnicas clave (2–5 bullets).  
                7.Recordatorio de seguridad (una línea).  

                Normas de estilo 
                - Siempre responde en español.  
                - Sé claro, estructurado y motivador.  
                - No inventes datos: pide al usuario lo que falte (objetivo, días, tiempo, material).  
                - Si el usuario pide algo fuera de alcance → responde:  
                Solo puedo ayudarte con planificación de rutinas de entrenamiento. ¿Quieres que te prepare una rutina adaptada a tus objetivos, tiempo y material""",
          }, 
          {"role": "user", 
            "content": pregunta #aqui estaba puesto antes "pregunta"
          }
      ],
      model="openai/gpt-oss-20b",
      stream=False,
  )

  return(chat_completion.choices[0].message.content)



In [11]:
pregunta = "Quiero que me des una rutina de entrenamiento para una persona que va a comenzar a sair a correr en calle, cuyo objetivo es correr una carrera de 10km dentro de 3 meses. Además tiene disponibilidad de entrenar 3 días por semana (aproximadamente 1 hora cada día)"

resp_llm = llm(pregunta)

print(resp_llm)

**1. Resumen del plan**  
- **Objetivo**: Correr una carrera de 10 km en 3 meses (12 semanas).  
- **Frecuencia**: 3 días por semana.  
- **Duración de cada sesión**: ~60 min (incluye calentamiento y estiramiento).  
- **Nivel inicial**: Principiante sin experiencia previa en carrera a pie.  
- **Material**: Sólo zapatillas de running y ropa cómoda.  

**2. Calendario semanal (ejemplo)**  

| Semana | Lunes | Miércoles | Viernes |
|--------|-------|-----------|---------|
| 1 | Run‑Walk 20 min | Tempo (poco más lento) 20 min | Long run 25 min |
| 2 | Run‑Walk 25 min | Tempo 25 min | Long run 30 min |
| 3 | Run‑Walk 30 min | Tempo 30 min | Long run 35 min |
| 4 | Run‑Walk 35 min | Tempo 35 min | Long run 40 min |
| 5 | Run‑Walk 40 min | Tempo 40 min | Long run 45 min |
| 6 | Run‑Walk 45 min | Tempo 45 min | Long run 50 min |
| 7 | Run‑Walk 50 min | Tempo 50 min | Long run 55 min |
| 8 | Run‑Walk 55 min | Tempo 55 min | Long run 60 min |
| 9 | Run‑Walk 60 min | Tempo 60 min | Long run 65 

In [14]:
pregunta = "Rutina de gimnasio cuyo objetivo es ganar masa muscular. rutina de 4 días por semana con una duración aproximada de 75 min cada entrenamiento. Entreno en un gimnasio comercial (suele tener todo tipo de maquina de ntrenamiento). objetivo ganas masa mulcular. Nivel intermedio. Sin limtaciones físicas. Con todo tipo de equipos para entrenar y sin preferencias"

respuesta = llm(pregunta)
respuesta_bbdd = bbdd(pregunta, respuesta)

if respuesta_bbdd == "Ok":
    print(pregunta)
    print("------------")
    print(respuesta)
else:
    print("Error")

Rutina de gimnasio cuyo objetivo es ganar masa muscular. rutina de 4 días por semana con una duración aproximada de 75 min cada entrenamiento. Entreno en un gimnasio comercial (suele tener todo tipo de maquina de ntrenamiento). objetivo ganas masa mulcular. Nivel intermedio. Sin limtaciones físicas. Con todo tipo de equipos para entrenar y sin preferencias
------------
**1. Resumen del plan**  
- **Objetivo**: Ganar masa muscular (hipertrofia).  
- **Frecuencia**: 4 días por semana (Lunes, Miércoles, Viernes, Sábado).  
- **Duración**: 75 min por entrenamiento (incluye calentamiento y estiramientos).  

---

**2. Calendario semanal**  

| Día | Enfoque principal | Zona |  
|-----|-------------------|------|  
| Lunes | Pecho + Tríceps | Pectorales y músculos antagonistas |  
| Miércoles | Espalda + Bíceps | Dorsales y músculos antagonistas |  
| Viernes | Piernas + Core | Cuádriceps, glúteos, isquiotibiales + abdomen |  
| Sábado | Hombros + Core | Deltoides + músculos estabilizadores 